# National Stop Place Register - Stop places to quays
* API - <https://developer.entur.org/stop-places-v1-read>




In [176]:
import pandas as pd
import requests

In [177]:
base_url = "https://api.entur.io/stop-places/v1/read"

## Stop places
* Returns deeply nested json. Need to flatten a bit.
* Best to read json and use json_normalize a bit.

Reading all stop-places takes 30 seconds, hence we need to cache for making use of this in an application

In [178]:
resp = requests.get(f"{base_url}/stop-places?count=140000")
df_flat = pd.json_normalize(resp.json()).convert_dtypes("pyarrow")
print(f"Number of stop places: {len(df_flat)}")
df_flat.dtypes

Number of stop places: 63346


id                                                                                                  string[python]
created                                                                                             string[python]
changed                                                                                             string[python]
modification                                                                                        string[python]
version                                                                                             string[python]
status_BasicModificationDetailsGroup                                                                string[python]
publication                                                                                         string[python]
transportMode                                                                                       string[python]
stopPlaceType                                                                   

A stop-place can contain one or more quays. These are in a list in `quays.quayRefOrQuay` so we use `explode` to get one row per quay.

In [179]:
df_exploded = (df_flat
 .explode("quays.quayRefOrQuay")
 .reset_index(drop=True)
)
df_exploded

,id,created,changed,modification,version,status_BasicModificationDetailsGroup,publication,transportMode,stopPlaceType,weighting,...,alternativeNames.alternativeName,waterSubmode,adjacentSites.modificationSet,adjacentSites.siteRef,metroSubmode,tramSubmode,railSubmode,telecabinSubmode,privateCode.value,airSubmode
0,NSR:StopPlace:1,2017-06-19T19:12:27.003+02:00,2024-02-05T18:36:34.461+01:00,NEW,5,ACTIVE,PUBLIC,RAIL,RAIL_STATION,INTERCHANGE_ALLOWED,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,NSR:StopPlace:1,2017-06-19T19:12:27.003+02:00,2024-02-05T18:36:34.461+01:00,NEW,5,ACTIVE,PUBLIC,RAIL,RAIL_STATION,INTERCHANGE_ALLOWED,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,NSR:StopPlace:10,2017-06-19T19:12:31.003+02:00,2022-02-18T14:14:31.041+01:00,NEW,6,ACTIVE,PUBLIC,BUS,ONSTREET_BUS,INTERCHANGE_ALLOWED,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,NSR:StopPlace:100,2017-06-19T19:12:36.685+02:00,2019-10-25T08:44:26.07+02:00,NEW,3,ACTIVE,PUBLIC,BUS,ONSTREET_BUS,INTERCHANGE_ALLOWED,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,NSR:StopPlace:1000,2017-06-19T19:14:34.1+02:00,2024-01-01T01:24:58.536+01:00,NEW,1,ACTIVE,PUBLIC,BUS,ONSTREET_BUS,INTERCHANGE_ALLOWED,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109239,NSR:StopPlace:9997,2017-06-19T19:24:52.79+02:00,2024-01-01T02:14:32.377+01:00,NEW,2,ACTIVE,PUBLIC,BUS,ONSTREET_BUS,INTERCHANGE_ALLOWED,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
109240,NSR:StopPlace:9998,2017-06-19T19:24:52.825+02:00,2024-01-01T02:14:32.42+01:00,NEW,4,ACTIVE,PUBLIC,BUS,ONSTREET_BUS,INTERCHANGE_ALLOWED,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
109241,NSR:StopPlace:9998,2017-06-19T19:24:52.825+02:00,2024-01-01T02:14:32.42+01:00,NEW,4,ACTIVE,PUBLIC,BUS,ONSTREET_BUS,INTERCHANGE_ALLOWED,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
109242,NSR:StopPlace:9999,2017-06-19T19:24:52.886+02:00,2024-01-01T02:14:32.467+01:00,NEW,2,ACTIVE,PUBLIC,BUS,ONSTREET_BUS,INTERCHANGE_ALLOWED,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


We need to flatten json for quays to get that data like centroid and name

In [180]:
df_quays = (pd.merge(
    df_exploded, 
    (
        pd.json_normalize(df_exploded["quays.quayRefOrQuay"])
        .set_index(df_exploded.index)
        .add_prefix("quay.")
    ),
    left_index=True,
    right_index=True,
    # suffix=("", "_quay")
)
.drop(columns="quays.quayRefOrQuay")
)
df_quays.sample(10).T.head(60)

,81968,15137,102563,69724,49372,15428,108113,4184,78526,41035
id,NSR:StopPlace:52345,NSR:StopPlace:17299,NSR:StopPlace:6815,NSR:StopPlace:45276,NSR:StopPlace:34876,NSR:StopPlace:17449,NSR:StopPlace:9453,NSR:StopPlace:12012,NSR:StopPlace:50369,NSR:StopPlace:30829
created,2017-06-19T20:04:39.5+02:00,2017-06-19T19:33:40.683+02:00,2017-06-19T19:20:03.391+02:00,2017-06-19T19:58:14.39+02:00,2017-06-19T19:48:57.594+02:00,2017-06-19T19:34:19.72+02:00,2017-06-19T19:24:27.314+02:00,2017-06-19T19:28:40.842+02:00,2017-06-19T20:02:18.517+02:00,2017-06-19T19:45:11.907+02:00
changed,2024-03-13T22:28:26.685+01:00,2024-03-13T21:59:27.126+01:00,2024-01-01T02:11:47.717+01:00,2024-01-01T01:54:56.471+01:00,2024-01-01T01:46:55.601+01:00,2024-01-01T01:31:56.176+01:00,2024-01-01T02:14:06.871+01:00,2024-01-01T01:27:04.574+01:00,2024-01-01T01:58:18.873+01:00,2024-02-05T18:56:17.474+01:00
modification,NEW,NEW,NEW,NEW,NEW,NEW,NEW,NEW,NEW,NEW
version,3,2,4,5,1,4,4,2,4,19
status_BasicModificationDetailsGroup,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE
publication,PUBLIC,PUBLIC,PUBLIC,PUBLIC,PUBLIC,PUBLIC,PUBLIC,PUBLIC,PUBLIC,PUBLIC
transportMode,BUS,BUS,BUS,BUS,BUS,BUS,BUS,BUS,BUS,BUS
stopPlaceType,ONSTREET_BUS,ONSTREET_BUS,ONSTREET_BUS,ONSTREET_BUS,ONSTREET_BUS,ONSTREET_BUS,ONSTREET_BUS,ONSTREET_BUS,ONSTREET_BUS,ONSTREET_BUS
weighting,INTERCHANGE_ALLOWED,INTERCHANGE_ALLOWED,INTERCHANGE_ALLOWED,INTERCHANGE_ALLOWED,INTERCHANGE_ALLOWED,INTERCHANGE_ALLOWED,INTERCHANGE_ALLOWED,INTERCHANGE_ALLOWED,INTERCHANGE_ALLOWED,INTERCHANGE_ALLOWED


In [185]:
df_quays.to_parquet("../data/quays.parquet")

In [187]:
pd.read_parquet("../data/quays.parquet")

,id,created,changed,modification,version,status_BasicModificationDetailsGroup,publication,transportMode,stopPlaceType,weighting,...,quay.privateCode.type,quay.description.value,quay.placeEquipments.id,quay.placeEquipments.modificationSet,quay.placeEquipments.installedEquipmentRefOrInstalledEquipment,quay.created,quay.name.value,quay.name.lang,quay.boardingPositions.modificationSet,quay.boardingPositions.boardingPositionRefOrBoardingPosition
0,NSR:StopPlace:1,2017-06-19T19:12:27.003+02:00,2024-02-05T18:36:34.461+01:00,NEW,5,ACTIVE,PUBLIC,RAIL,RAIL_STATION,INTERCHANGE_ALLOWED,...,None,None,None,None,None,None,None,None,None,None
1,NSR:StopPlace:1,2017-06-19T19:12:27.003+02:00,2024-02-05T18:36:34.461+01:00,NEW,5,ACTIVE,PUBLIC,RAIL,RAIL_STATION,INTERCHANGE_ALLOWED,...,None,None,None,None,None,None,None,None,None,None
2,NSR:StopPlace:10,2017-06-19T19:12:31.003+02:00,2022-02-18T14:14:31.041+01:00,NEW,6,ACTIVE,PUBLIC,BUS,ONSTREET_BUS,INTERCHANGE_ALLOWED,...,None,None,None,None,None,None,None,None,None,None
3,NSR:StopPlace:100,2017-06-19T19:12:36.685+02:00,2019-10-25T08:44:26.07+02:00,NEW,3,ACTIVE,PUBLIC,BUS,ONSTREET_BUS,INTERCHANGE_ALLOWED,...,None,None,None,None,None,None,None,None,None,None
4,NSR:StopPlace:1000,2017-06-19T19:14:34.1+02:00,2024-01-01T01:24:58.536+01:00,NEW,1,ACTIVE,PUBLIC,BUS,ONSTREET_BUS,INTERCHANGE_ALLOWED,...,StopPoint,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109239,NSR:StopPlace:9997,2017-06-19T19:24:52.79+02:00,2024-01-01T02:14:32.377+01:00,NEW,2,ACTIVE,PUBLIC,BUS,ONSTREET_BUS,INTERCHANGE_ALLOWED,...,None,mot Øyermoen,NSR:PlaceEquipment:48070,ALL,"[{'type': 'GeneralSign', 'value': {'content': ...",None,None,None,None,None
109240,NSR:StopPlace:9998,2017-06-19T19:24:52.825+02:00,2024-01-01T02:14:32.42+01:00,NEW,4,ACTIVE,PUBLIC,BUS,ONSTREET_BUS,INTERCHANGE_ALLOWED,...,None,None,None,None,None,None,None,None,None,None
109241,NSR:StopPlace:9998,2017-06-19T19:24:52.825+02:00,2024-01-01T02:14:32.42+01:00,NEW,4,ACTIVE,PUBLIC,BUS,ONSTREET_BUS,INTERCHANGE_ALLOWED,...,None,None,None,None,None,None,None,None,None,None
109242,NSR:StopPlace:9999,2017-06-19T19:24:52.886+02:00,2024-01-01T02:14:32.467+01:00,NEW,2,ACTIVE,PUBLIC,BUS,ONSTREET_BUS,INTERCHANGE_ALLOWED,...,None,mot Berg,NSR:PlaceEquipment:48067,ALL,"[{'type': 'GeneralSign', 'value': {'content': ...",None,None,None,None,None
